# Day 01: Bayes' Theorem - Foundations for Quantitative Trading

## Learning Objectives
- Understand Bayes' Theorem and its components
- Master the concepts of Prior, Likelihood, and Posterior
- Apply Bayesian reasoning to trading scenarios
- Build intuition for updating beliefs with new market evidence

---

## 1. Bayes' Theorem: The Foundation

### The Formula

$$P(H|E) = \frac{P(E|H) \cdot P(H)}{P(E)}$$

Where:
- **P(H|E)** = **Posterior** - Probability of hypothesis H given evidence E
- **P(E|H)** = **Likelihood** - Probability of evidence E given hypothesis H is true
- **P(H)** = **Prior** - Initial probability of hypothesis H before seeing evidence
- **P(E)** = **Marginal Likelihood** (Evidence) - Total probability of observing E

### In Trading Terms

$$P(\text{Market Up}|\text{Signal}) = \frac{P(\text{Signal}|\text{Market Up}) \cdot P(\text{Market Up})}{P(\text{Signal})}$$

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns

plt.style.use('seaborn-v0_8-whitegrid')
np.random.seed(42)

print("Bayesian Methods for Quantitative Trading")
print("=" * 45)

## 2. Understanding the Components

### 2.1 Prior: Your Initial Belief

The prior represents what you believe **before** seeing new data.

**Trading Examples:**
- Historical win rate of a strategy (e.g., 55% win rate)
- Base probability of market going up on any given day (~53%)
- Prior belief about a stock's volatility regime

In [ ]:
# Example: Prior beliefs about market direction

# Historical data: S&P 500 up days
historical_up_days = 0.53  # ~53% of days are positive historically
historical_down_days = 1 - historical_up_days

print("PRIOR BELIEFS (Before seeing today's data)")
print("-" * 45)
print(f"P(Market Up)   = {historical_up_days:.2%}")
print(f"P(Market Down) = {historical_down_days:.2%}")

# Visualize prior
fig, ax = plt.subplots(figsize=(8, 4))
categories = ['Market Up', 'Market Down']
priors = [historical_up_days, historical_down_days]
colors = ['green', 'red']
ax.bar(categories, priors, color=colors, alpha=0.7, edgecolor='black')
ax.set_ylabel('Probability')
ax.set_title('Prior Distribution: Historical Market Direction')
ax.set_ylim(0, 1)
for i, v in enumerate(priors):
    ax.text(i, v + 0.02, f'{v:.1%}', ha='center', fontweight='bold')
plt.tight_layout()
plt.show()

### 2.2 Likelihood: How Probable is the Evidence?

The likelihood tells us: **Given a hypothesis is true, how likely is this evidence?**

**Trading Examples:**
- If market will go up, what's the probability of seeing a bullish technical signal?
- If a stock will beat earnings, what's the probability of insider buying?
- If volatility will spike, what's the probability of seeing VIX futures in backwardation?

In [ ]:
# Example: Likelihood of seeing a bullish signal

# Define our signal: "Golden Cross" (50-day MA crosses above 200-day MA)
# Based on historical backtesting:

# P(Golden Cross | Market will go up) - signal appears before up moves
p_signal_given_up = 0.70  # 70% of up moves were preceded by golden cross

# P(Golden Cross | Market will go down) - false signals
p_signal_given_down = 0.25  # 25% of down moves also had golden cross (false positives)

print("LIKELIHOOD (Signal behavior given outcomes)")
print("-" * 45)
print(f"P(Golden Cross | Market Up)   = {p_signal_given_up:.2%}")
print(f"P(Golden Cross | Market Down) = {p_signal_given_down:.2%}")
print()
print("Interpretation:")
print("- When market goes up, we usually see the signal beforehand (70%)")
print("- When market goes down, we sometimes still see the signal (25% - false positives)")

### 2.3 Marginal Likelihood (Evidence): Normalizing Factor

The marginal likelihood is the **total probability of seeing the evidence** across all hypotheses.

$$P(E) = P(E|H) \cdot P(H) + P(E|\neg H) \cdot P(\neg H)$$

In [ ]:
# Calculate P(Golden Cross) - total probability of seeing the signal

p_signal = (p_signal_given_up * historical_up_days + 
            p_signal_given_down * historical_down_days)

print("MARGINAL LIKELIHOOD (Total probability of signal)")
print("-" * 45)
print(f"P(Golden Cross) = P(GC|Up)×P(Up) + P(GC|Down)×P(Down)")
print(f"P(Golden Cross) = {p_signal_given_up:.2f}×{historical_up_days:.2f} + {p_signal_given_down:.2f}×{historical_down_days:.2f}")
print(f"P(Golden Cross) = {p_signal_given_up * historical_up_days:.4f} + {p_signal_given_down * historical_down_days:.4f}")
print(f"P(Golden Cross) = {p_signal:.4f} = {p_signal:.2%}")

### 2.4 Posterior: Updated Belief

The posterior is our **updated probability after incorporating new evidence**.

This is what we care about as traders: **Given that we see a signal, what's the probability the trade will be profitable?**

In [ ]:
# Apply Bayes' Theorem
# P(Market Up | Golden Cross) = P(Golden Cross | Market Up) × P(Market Up) / P(Golden Cross)

p_up_given_signal = (p_signal_given_up * historical_up_days) / p_signal
p_down_given_signal = (p_signal_given_down * historical_down_days) / p_signal

print("POSTERIOR (Updated belief after seeing signal)")
print("=" * 50)
print()
print("Bayes' Theorem:")
print(f"P(Up|Signal) = P(Signal|Up) × P(Up) / P(Signal)")
print(f"P(Up|Signal) = {p_signal_given_up:.2f} × {historical_up_days:.2f} / {p_signal:.4f}")
print(f"P(Up|Signal) = {p_up_given_signal:.4f}")
print()
print("-" * 50)
print(f"PRIOR:     P(Market Up) = {historical_up_days:.2%}")
print(f"POSTERIOR: P(Market Up | Golden Cross) = {p_up_given_signal:.2%}")
print("-" * 50)
print(f"\n✓ Signal improves our confidence by {(p_up_given_signal - historical_up_days):.2%}")
print(f"✓ Posterior odds: {p_up_given_signal/p_down_given_signal:.2f}:1 in favor of up move")

In [ ]:
# Visualize Prior vs Posterior

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Prior
axes[0].bar(['Up', 'Down'], [historical_up_days, historical_down_days], 
            color=['green', 'red'], alpha=0.7, edgecolor='black')
axes[0].set_ylabel('Probability')
axes[0].set_title('PRIOR\n(Before seeing signal)', fontsize=12, fontweight='bold')
axes[0].set_ylim(0, 1)
axes[0].axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)
for i, v in enumerate([historical_up_days, historical_down_days]):
    axes[0].text(i, v + 0.02, f'{v:.1%}', ha='center', fontweight='bold')

# Posterior
axes[1].bar(['Up', 'Down'], [p_up_given_signal, p_down_given_signal], 
            color=['green', 'red'], alpha=0.7, edgecolor='black')
axes[1].set_ylabel('Probability')
axes[1].set_title('POSTERIOR\n(After seeing Golden Cross)', fontsize=12, fontweight='bold')
axes[1].set_ylim(0, 1)
axes[1].axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)
for i, v in enumerate([p_up_given_signal, p_down_given_signal]):
    axes[1].text(i, v + 0.02, f'{v:.1%}', ha='center', fontweight='bold')

plt.suptitle('Bayesian Update: How a Signal Changes Our Beliefs', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

---

## 3. Trading Example: Earnings Signal Analysis

**Scenario:** You're analyzing whether to buy a stock before earnings based on analyst sentiment.

In [ ]:
class BayesianEarningsAnalyzer:
    """Bayesian analysis for earnings trading signals."""
    
    def __init__(self, prior_beat_rate=0.65):
        """
        Initialize with prior belief about earnings beat rate.
        
        Parameters:
        -----------
        prior_beat_rate : float
            Historical probability that companies beat earnings (~65% on average)
        """
        self.prior_beat = prior_beat_rate
        self.prior_miss = 1 - prior_beat_rate
        
    def update_belief(self, signal_name, p_signal_given_beat, p_signal_given_miss):
        """
        Update belief using Bayes' theorem.
        
        Parameters:
        -----------
        signal_name : str
            Name of the signal
        p_signal_given_beat : float
            P(Signal | Beat) - likelihood of signal when company beats
        p_signal_given_miss : float
            P(Signal | Miss) - likelihood of signal when company misses
        """
        # Marginal likelihood
        p_signal = (p_signal_given_beat * self.prior_beat + 
                   p_signal_given_miss * self.prior_miss)
        
        # Posterior
        posterior_beat = (p_signal_given_beat * self.prior_beat) / p_signal
        posterior_miss = (p_signal_given_miss * self.prior_miss) / p_signal
        
        # Results
        result = {
            'signal': signal_name,
            'prior_beat': self.prior_beat,
            'posterior_beat': posterior_beat,
            'improvement': posterior_beat - self.prior_beat,
            'posterior_odds': posterior_beat / posterior_miss,
            'likelihood_ratio': p_signal_given_beat / p_signal_given_miss
        }
        
        return result
    
    def analyze_multiple_signals(self, signals):
        """Analyze multiple independent signals."""
        results = []
        for signal in signals:
            result = self.update_belief(
                signal['name'],
                signal['p_given_beat'],
                signal['p_given_miss']
            )
            results.append(result)
        return pd.DataFrame(results)


# Initialize analyzer
analyzer = BayesianEarningsAnalyzer(prior_beat_rate=0.65)

# Define signals and their likelihoods
signals = [
    {
        'name': 'Analyst Upgrades (>3)',
        'p_given_beat': 0.75,  # 75% of beats had multiple upgrades
        'p_given_miss': 0.20   # Only 20% of misses had upgrades
    },
    {
        'name': 'Insider Buying',
        'p_given_beat': 0.45,  # 45% of beats had insider buying
        'p_given_miss': 0.15   # 15% of misses had insider buying
    },
    {
        'name': 'High Short Interest',
        'p_given_beat': 0.25,  # 25% of beats had high short interest
        'p_given_miss': 0.55   # 55% of misses had high short interest
    },
    {
        'name': 'Options IV Spike',
        'p_given_beat': 0.60,  # 60% of beats had IV spike
        'p_given_miss': 0.50   # 50% of misses also had IV spike (less informative)
    }
]

# Analyze all signals
results_df = analyzer.analyze_multiple_signals(signals)

print("BAYESIAN SIGNAL ANALYSIS: Earnings Prediction")
print("=" * 70)
print(f"Prior P(Beat) = {analyzer.prior_beat:.1%}")
print()
print(results_df.to_string(index=False, float_format='{:.2%}'.format))

In [ ]:
# Visualize signal effectiveness

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Prior vs Posterior comparison
x = np.arange(len(results_df))
width = 0.35

bars1 = axes[0].bar(x - width/2, results_df['prior_beat'], width, 
                    label='Prior', color='lightblue', edgecolor='black')
bars2 = axes[0].bar(x + width/2, results_df['posterior_beat'], width,
                    label='Posterior', color='darkblue', edgecolor='black')

axes[0].set_xlabel('Signal')
axes[0].set_ylabel('P(Beat)')
axes[0].set_title('Prior vs Posterior Probability of Earnings Beat')
axes[0].set_xticks(x)
axes[0].set_xticklabels([s[:15] + '...' if len(s) > 15 else s for s in results_df['signal']], 
                        rotation=45, ha='right')
axes[0].legend()
axes[0].axhline(y=0.65, color='red', linestyle='--', alpha=0.5, label='Prior')
axes[0].set_ylim(0, 1)

# Right: Likelihood Ratio (signal strength)
colors = ['green' if lr > 1 else 'red' for lr in results_df['likelihood_ratio']]
axes[1].barh(results_df['signal'], results_df['likelihood_ratio'], color=colors, 
             alpha=0.7, edgecolor='black')
axes[1].axvline(x=1, color='black', linestyle='--', linewidth=2)
axes[1].set_xlabel('Likelihood Ratio (>1 = Bullish Signal)')
axes[1].set_title('Signal Strength: Likelihood Ratio\nP(Signal|Beat) / P(Signal|Miss)')

# Add annotations
for i, (lr, sig) in enumerate(zip(results_df['likelihood_ratio'], results_df['signal'])):
    axes[1].text(lr + 0.1, i, f'{lr:.2f}x', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("\nKey Insights:")
print("-" * 50)
print("• Likelihood Ratio > 1: Signal is bullish (increases P(Beat))")
print("• Likelihood Ratio < 1: Signal is bearish (decreases P(Beat))")
print("• Likelihood Ratio = 1: Signal is uninformative")

---

## 4. Sequential Bayesian Updates: Combining Multiple Signals

One of the most powerful aspects of Bayesian inference is **sequential updating** - we can incorporate multiple pieces of evidence one at a time, with each posterior becoming the prior for the next update.

In [ ]:
def sequential_bayesian_update(prior, signals_observed):
    """
    Perform sequential Bayesian updates with multiple signals.
    
    Parameters:
    -----------
    prior : float
        Initial prior probability
    signals_observed : list of tuples
        Each tuple: (signal_name, p_signal_given_H, p_signal_given_not_H)
    
    Returns:
    --------
    list of dicts with update history
    """
    history = [{'step': 0, 'signal': 'Prior', 'probability': prior}]
    current_prob = prior
    
    for i, (name, p_given_h, p_given_not_h) in enumerate(signals_observed, 1):
        # Marginal likelihood
        p_signal = p_given_h * current_prob + p_given_not_h * (1 - current_prob)
        
        # Posterior becomes new prior
        current_prob = (p_given_h * current_prob) / p_signal
        
        history.append({
            'step': i,
            'signal': name,
            'probability': current_prob
        })
    
    return history


# Example: Trading decision with multiple signals
print("SEQUENTIAL BAYESIAN UPDATE: Stock Analysis")
print("=" * 60)
print("\nScenario: Deciding whether to go LONG before earnings\n")

# Start with base rate
prior_profit = 0.50  # 50-50 chance without any signal

# Signals we observe (in order)
observed_signals = [
    ('Strong Revenue Growth (>20%)', 0.80, 0.30),  # Bullish
    ('3 Analyst Upgrades', 0.75, 0.25),            # Bullish
    ('CEO Sold Shares', 0.20, 0.45),              # Bearish
    ('Sector Momentum Positive', 0.70, 0.40),      # Mildly Bullish
]

history = sequential_bayesian_update(prior_profit, observed_signals)

# Display results
print("Update History:")
print("-" * 60)
for h in history:
    arrow = "" if h['step'] == 0 else "→ "
    print(f"Step {h['step']}: {arrow}{h['signal']:<35} P(Profit) = {h['probability']:.2%}")

print("\n" + "=" * 60)
print(f"FINAL POSTERIOR: P(Profitable Trade) = {history[-1]['probability']:.2%}")

In [ ]:
# Visualize sequential updates

fig, ax = plt.subplots(figsize=(12, 6))

steps = [h['step'] for h in history]
probs = [h['probability'] for h in history]
labels = [h['signal'] for h in history]

# Plot line and points
ax.plot(steps, probs, 'b-o', linewidth=2, markersize=10, markerfacecolor='white', 
        markeredgewidth=2, markeredgecolor='blue')

# Fill areas
ax.fill_between(steps, probs, 0.5, where=[p > 0.5 for p in probs], 
                color='green', alpha=0.3, label='Above neutral')
ax.fill_between(steps, probs, 0.5, where=[p < 0.5 for p in probs], 
                color='red', alpha=0.3, label='Below neutral')

# Reference lines
ax.axhline(y=0.5, color='gray', linestyle='--', linewidth=1, label='Neutral (50%)')
ax.axhline(y=0.7, color='green', linestyle=':', alpha=0.5, label='Strong Buy (70%)')
ax.axhline(y=0.3, color='red', linestyle=':', alpha=0.5, label='Strong Sell (30%)')

# Annotations
for i, (step, prob, label) in enumerate(zip(steps, probs, labels)):
    offset = 0.05 if prob > 0.5 else -0.08
    ax.annotate(f'{prob:.1%}\n{label}', xy=(step, prob), 
                xytext=(step, prob + offset),
                ha='center', fontsize=9,
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

ax.set_xlabel('Update Step', fontsize=12)
ax.set_ylabel('P(Profitable Trade)', fontsize=12)
ax.set_title('Sequential Bayesian Update: How Each Signal Changes Our Belief', 
             fontsize=14, fontweight='bold')
ax.set_ylim(0, 1)
ax.set_xticks(steps)
ax.legend(loc='lower right')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 5. Continuous Priors: Beta Distribution for Win Rate Estimation

In practice, we often use **continuous distributions** as priors. The **Beta distribution** is perfect for modeling probabilities (like win rates).

In [ ]:
class BayesianWinRateEstimator:
    """
    Bayesian estimation of trading strategy win rate using Beta-Binomial model.
    
    Beta prior + Binomial likelihood = Beta posterior (conjugate prior)
    """
    
    def __init__(self, alpha_prior=1, beta_prior=1):
        """
        Initialize with Beta prior.
        
        Parameters:
        -----------
        alpha_prior : float
            Prior "pseudo-wins" (α parameter)
        beta_prior : float
            Prior "pseudo-losses" (β parameter)
            
        Common priors:
        - Beta(1,1): Uniform/uninformative - no prior belief
        - Beta(2,2): Slight belief in 50% win rate
        - Beta(10,10): Strong belief in 50% win rate
        - Beta(55,45): Prior belief of 55% win rate
        """
        self.alpha = alpha_prior
        self.beta = beta_prior
        self.wins = 0
        self.losses = 0
        self.history = []
        
    def update(self, wins, losses):
        """Update posterior with new trade results."""
        self.wins += wins
        self.losses += losses
        self.history.append({
            'total_wins': self.wins,
            'total_losses': self.losses,
            'alpha': self.alpha + self.wins,
            'beta': self.beta + self.losses
        })
        
    @property
    def posterior_alpha(self):
        return self.alpha + self.wins
    
    @property
    def posterior_beta(self):
        return self.beta + self.losses
    
    def mean(self):
        """Posterior mean (expected win rate)."""
        return self.posterior_alpha / (self.posterior_alpha + self.posterior_beta)
    
    def credible_interval(self, confidence=0.95):
        """Bayesian credible interval for win rate."""
        lower = (1 - confidence) / 2
        upper = 1 - lower
        return (
            stats.beta.ppf(lower, self.posterior_alpha, self.posterior_beta),
            stats.beta.ppf(upper, self.posterior_alpha, self.posterior_beta)
        )
    
    def prob_above_threshold(self, threshold=0.5):
        """Probability that true win rate is above threshold."""
        return 1 - stats.beta.cdf(threshold, self.posterior_alpha, self.posterior_beta)
    
    def get_distribution(self):
        """Return the posterior Beta distribution."""
        return stats.beta(self.posterior_alpha, self.posterior_beta)


# Example: New trading strategy evaluation
print("BAYESIAN WIN RATE ESTIMATION")
print("=" * 50)

# Start with weak prior (near-uniform)
estimator = BayesianWinRateEstimator(alpha_prior=2, beta_prior=2)

# Simulate first month of trading: 12 wins, 8 losses
estimator.update(wins=12, losses=8)

print(f"\nAfter Month 1: {estimator.wins}W / {estimator.losses}L")
print(f"Posterior Mean Win Rate: {estimator.mean():.2%}")
print(f"95% Credible Interval: [{estimator.credible_interval()[0]:.2%}, {estimator.credible_interval()[1]:.2%}]")
print(f"P(Win Rate > 50%): {estimator.prob_above_threshold(0.5):.2%}")

In [ ]:
# Visualize how posterior evolves with more data

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
x = np.linspace(0, 1, 1000)

# Simulate cumulative trading results
monthly_results = [
    (12, 8),   # Month 1
    (14, 9),   # Month 2
    (11, 7),   # Month 3
    (15, 10),  # Month 4
]

# Fresh estimator
est = BayesianWinRateEstimator(alpha_prior=2, beta_prior=2)

# Prior
prior_dist = stats.beta(2, 2)
axes[0, 0].fill_between(x, prior_dist.pdf(x), alpha=0.5, color='blue')
axes[0, 0].axvline(x=0.5, color='red', linestyle='--', label='50% threshold')
axes[0, 0].set_title('Prior: Beta(2,2)\nWeak prior, slight belief in 50%', fontsize=11)
axes[0, 0].set_xlabel('Win Rate')
axes[0, 0].set_ylabel('Density')
axes[0, 0].legend()

# After each month
titles = [
    'After Month 1\n12W/8L (60% raw)',
    'After Month 2\n26W/17L (60.5% raw)',
    'After Month 3\n37W/24L (60.7% raw)'
]

for i, ((wins, losses), ax, title) in enumerate(zip(monthly_results[:3], 
                                                     [axes[0,1], axes[1,0], axes[1,1]], 
                                                     titles)):
    est.update(wins, losses)
    post_dist = est.get_distribution()
    
    # Plot
    ax.fill_between(x, post_dist.pdf(x), alpha=0.5, color='green')
    ax.axvline(x=0.5, color='red', linestyle='--', label='50% threshold')
    ax.axvline(x=est.mean(), color='black', linestyle='-', linewidth=2, 
               label=f'Mean: {est.mean():.1%}')
    
    # Credible interval
    ci_low, ci_high = est.credible_interval()
    ax.axvspan(ci_low, ci_high, alpha=0.2, color='yellow', label=f'95% CI')
    
    ax.set_title(f'{title}\nP(WR>50%)={est.prob_above_threshold():.1%}', fontsize=11)
    ax.set_xlabel('Win Rate')
    ax.set_ylabel('Density')
    ax.legend(fontsize=8)

plt.suptitle('Bayesian Win Rate Estimation: Posterior Shrinks with More Data', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("\nKey Insight: As we collect more data, our uncertainty (width of distribution) decreases.")
print("The posterior becomes more concentrated around the true win rate.")

---

## 6. Practical Application: When to Trust a Trading Signal?

Use Bayesian analysis to determine if a new trading signal is reliable.

In [ ]:
def analyze_signal_reliability(signal_name, wins, losses, 
                               prior_alpha=1, prior_beta=1,
                               min_confidence=0.90,
                               target_win_rate=0.55):
    """
    Determine if a trading signal is reliable using Bayesian analysis.
    
    Parameters:
    -----------
    signal_name : str
        Name of the signal
    wins : int
        Number of winning trades
    losses : int
        Number of losing trades
    prior_alpha, prior_beta : float
        Beta prior parameters
    min_confidence : float
        Minimum required P(win_rate > target)
    target_win_rate : float
        Target win rate threshold
    """
    total = wins + losses
    raw_win_rate = wins / total if total > 0 else 0
    
    # Posterior
    post_alpha = prior_alpha + wins
    post_beta = prior_beta + losses
    
    # Bayesian estimates
    posterior_mean = post_alpha / (post_alpha + post_beta)
    posterior_dist = stats.beta(post_alpha, post_beta)
    
    ci_low, ci_high = posterior_dist.ppf(0.025), posterior_dist.ppf(0.975)
    prob_above_target = 1 - posterior_dist.cdf(target_win_rate)
    
    # Decision
    is_reliable = prob_above_target >= min_confidence
    
    print(f"\n{'='*60}")
    print(f"SIGNAL RELIABILITY ANALYSIS: {signal_name}")
    print(f"{'='*60}")
    print(f"\nSample: {wins}W / {losses}L (n={total})")
    print(f"Raw Win Rate: {raw_win_rate:.2%}")
    print(f"\n--- Bayesian Estimates ---")
    print(f"Posterior Mean: {posterior_mean:.2%}")
    print(f"95% Credible Interval: [{ci_low:.2%}, {ci_high:.2%}]")
    print(f"\n--- Decision Criteria ---")
    print(f"Target: Win Rate > {target_win_rate:.0%}")
    print(f"Required Confidence: {min_confidence:.0%}")
    print(f"Actual Confidence: {prob_above_target:.2%}")
    print(f"\n{'='*60}")
    
    if is_reliable:
        print(f"✅ DECISION: SIGNAL IS RELIABLE")
        print(f"   We are {prob_above_target:.0%} confident that true win rate > {target_win_rate:.0%}")
    else:
        print(f"⚠️  DECISION: NEED MORE DATA")
        print(f"   Only {prob_above_target:.0%} confident, need {min_confidence:.0%}")
        
        # How many more trades needed?
        for n in range(total, total + 500, 10):
            # Assume future trades maintain same win rate
            future_wins = int(n * raw_win_rate)
            future_losses = n - future_wins
            future_alpha = prior_alpha + future_wins
            future_beta = prior_beta + future_losses
            future_prob = 1 - stats.beta.cdf(target_win_rate, future_alpha, future_beta)
            if future_prob >= min_confidence:
                print(f"   → Need ~{n - total} more trades at current win rate to reach {min_confidence:.0%} confidence")
                break
    
    return is_reliable, prob_above_target


# Example analyses
analyze_signal_reliability("RSI Divergence Signal", wins=35, losses=25)
analyze_signal_reliability("New Experimental Signal", wins=8, losses=5)

---

## 7. Key Takeaways

### Bayes' Theorem Summary

| Component | Formula | Trading Interpretation |
|-----------|---------|------------------------|
| **Prior** P(H) | Initial belief | Historical win rate, market base rate |
| **Likelihood** P(E\|H) | P(evidence \| hypothesis) | How often signal appears before wins |
| **Marginal** P(E) | Total evidence probability | Overall frequency of signal |
| **Posterior** P(H\|E) | Updated belief | Probability of profit given signal |

### Practical Applications in Trading

1. **Signal Validation**: Determine if a trading signal actually has edge
2. **Strategy Evaluation**: Estimate true win rate with uncertainty bounds
3. **Combining Information**: Update beliefs as new market data arrives
4. **Risk Management**: Make decisions based on probability distributions, not point estimates
5. **Regime Detection**: Update beliefs about market regime based on evidence

In [ ]:
# Summary: Bayes' Theorem Calculator

def bayes_calculator(prior_h, likelihood_e_given_h, likelihood_e_given_not_h):
    """
    Simple Bayes' Theorem calculator.
    
    Parameters:
    -----------
    prior_h : float
        P(H) - Prior probability of hypothesis
    likelihood_e_given_h : float
        P(E|H) - Probability of evidence given H is true
    likelihood_e_given_not_h : float
        P(E|¬H) - Probability of evidence given H is false
    
    Returns:
    --------
    float : P(H|E) - Posterior probability
    """
    prior_not_h = 1 - prior_h
    marginal_e = likelihood_e_given_h * prior_h + likelihood_e_given_not_h * prior_not_h
    posterior = (likelihood_e_given_h * prior_h) / marginal_e
    
    return posterior


# Interactive example
print("BAYES' THEOREM CALCULATOR")
print("=" * 40)
print("\nExample: Will the market go up tomorrow?")
print()

# Inputs
prior = 0.53               # Historical up day probability
p_signal_given_up = 0.75   # P(bullish signal | market up)
p_signal_given_down = 0.30 # P(bullish signal | market down)

# Calculate
posterior = bayes_calculator(prior, p_signal_given_up, p_signal_given_down)

print(f"Prior P(Up): {prior:.2%}")
print(f"P(Bullish Signal | Up): {p_signal_given_up:.2%}")
print(f"P(Bullish Signal | Down): {p_signal_given_down:.2%}")
print()
print(f"→ Posterior P(Up | Bullish Signal): {posterior:.2%}")
print(f"→ Edge gained from signal: +{(posterior - prior):.2%}")

---

## 8. Practice Exercises

### Exercise 1: Signal Analysis
A mean reversion signal fires 40% of the time when a stock rebounds (profitable), but also fires 15% of the time when it continues falling (loss). If stocks historically rebound 35% of the time after a 5% drop, what's the posterior probability of rebound given the signal?

### Exercise 2: Strategy Evaluation  
Your new strategy has 23 wins and 17 losses. Using a Beta(2,2) prior:
- What is the posterior mean win rate?
- What is the 95% credible interval?
- What is P(win rate > 55%)?

### Exercise 3: Sequential Updates
Start with P(Bull Market) = 0.60. Update sequentially with:
1. GDP growth positive: P(positive GDP | Bull) = 0.85, P(positive GDP | Bear) = 0.40
2. Yield curve inverts: P(inversion | Bull) = 0.15, P(inversion | Bear) = 0.60

What is the final posterior P(Bull Market)?

In [ ]:
# Exercise Solutions (uncomment to run)

# Exercise 1
print("Exercise 1: Signal Analysis")
print("-" * 40)
ex1_posterior = bayes_calculator(
    prior_h=0.35,              # P(Rebound)
    likelihood_e_given_h=0.40,  # P(Signal | Rebound)
    likelihood_e_given_not_h=0.15  # P(Signal | No Rebound)
)
print(f"P(Rebound | Signal) = {ex1_posterior:.2%}")
print()

# Exercise 2
print("Exercise 2: Strategy Evaluation")
print("-" * 40)
ex2_est = BayesianWinRateEstimator(alpha_prior=2, beta_prior=2)
ex2_est.update(wins=23, losses=17)
print(f"Posterior Mean: {ex2_est.mean():.2%}")
print(f"95% CI: {ex2_est.credible_interval()}")
print(f"P(WR > 55%): {ex2_est.prob_above_threshold(0.55):.2%}")
print()

# Exercise 3
print("Exercise 3: Sequential Updates")
print("-" * 40)
ex3_signals = [
    ('Positive GDP', 0.85, 0.40),
    ('Yield Curve Inverts', 0.15, 0.60)
]
ex3_history = sequential_bayesian_update(0.60, ex3_signals)
for h in ex3_history:
    print(f"{h['signal']}: P(Bull) = {h['probability']:.2%}")

---

## Next Steps

- **Day 02**: Conjugate Priors and Bayesian Linear Regression
- **Day 03**: Markov Chain Monte Carlo (MCMC) for Complex Models
- **Day 04**: Bayesian Portfolio Optimization
- **Day 05**: Bayesian Time Series and Regime Detection